# Process the Customers Data
  1. Ingest the data into the data lakehouse - bronze_customers
  2. Perform data quality checks and transform the data as required - silver_customers_clean
  3. Apply changes to the Customers data - silver_customers

### 1. Ingest the data into the data lakehouse - bronze_customers

![process_customers_data.png](./process_customers_data.png "process_customers_data.png")

In [0]:
create or refresh streaming table bronze_customers
comment 'Raw customers data ingested from the source system operational data'
tblproperties ('quality' = 'bronze')
as 
select *, 
        _metadata.file_path as input_file_path,
        current_timestamp() as ingestion_timestamp
from cloud_files(
  '/Volumes/circuitbox/landing/operational_data/customers/', 
  'json',
  map('cloudFiles.inferColumnTypes','true')
);

### 2. Perform data quality checks and transform the data as required - silver_customers_clean

![customers_data_silver_layer.png](./customers_data_silver_layer.png "customers_data_silver_layer.png")

In [0]:
create or refresh streaming table silver_customers_clean(
  constraint valid_customer_id expect(customer_id is not null) on violation fail update,
  constraint valid_customer_name expect(customer_name is not null) on violation drop row,
  constraint valid_telephone expect(length(telephone) >= 10),
  constraint valid_email expect(email is not null),
  constraint valid_date_of_birth expect(date_of_birth >= '1920-01-01')
)
comment 'Cleaned customers data'
tblproperties('quality' = 'silver')
as
select customer_id,
       customer_name,
       cast(date_of_birth as date) as date_of_birth,
       telephone,
       email,
       cast(created_date as date) as created_date
  from stream(live.bronze_customers)

### 3. Apply changes to the Customers data - silver_customers

![customers_data_scd_type1_silver_layer.png](./customers_data_scd_type1_silver_layer.png "customers_data_scd_type1_silver_layer.png")

In [0]:
create or refresh streaming table silver_customers
comment 'SCD Type 1 customers Data'
tblproperties('quality' = 'silver');

In [0]:
apply changes into live.silver_customers
from stream(live.silver_customers_clean)
keys (customer_id)
sequence by created_date
stored as SCD Type 1; -- Type1 is default value